<a href="https://colab.research.google.com/github/jennahgosciak/nyc_fire_risk/blob/main/00_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "2_intermediate")
out= os.join(root, "data", "3_clean")

In [161]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [162]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: hpd_agg_final.csv, ID: 1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: hpd_per_bbl.csv, ID: 1xXilfI1zEmyEYE_TtMdawcnmnZlTv__Z
Title: hpd_2012_2022_v3.csv, ID: 1HXst8E2ZuPVKvKe_9jmSQwKYppOJjraD
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp
Title: all_vacate.csv, ID: 1WfnTEjOaDNEb3E2vurXS_XkxJ0J1isdN
Title: pluto_ct.csv, ID: 1ZLZZS79fi4Z0RFTNMR

## Load outcomes (fire vacate orders)

In [163]:
csv = drive.CreateFile({'id': '1caDnIMowquAoJuZi1tDlyEqAS4HhXiU0'})
csv.GetContentFile('fire_vacate_bbl.csv')  

df= pd.read_csv('fire_vacate_bbl.csv')[["bbl", "num_vac_orders"]]
df["bbl"]= df["bbl"].astype(str).str.replace(".0", "", regex = False)

### Load PLUTO

In [164]:
csv = drive.CreateFile({'id': '1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o'})
csv.GetContentFile('pluto_df.csv')  

pluto= pd.read_csv('pluto_df.csv').drop("Unnamed: 0", axis =1)
pluto["bbl"]= pluto["bbl"].astype(str).str.replace(".0", "", regex = False)

C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [165]:
pluto["boroct2010"]= pluto["boroct2010"].astype(str)
pluto.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,bldgclass_RX,bldgclass_RZ,bldgclass_S0,bldgclass_S1,bldgclass_S2,bldgclass_S3,bldgclass_S4,bldgclass_S5,bldgclass_S9,bldgclass_Z0
0,MN,1012090027,107,181.0,10025,7 WEST 95 STREET,C2,2,NaN,ROBERT & ANNE RABBINO,...,0,0,0,0,0,0,0,0,0,0
1,MN,1003730016,103,2601.0,10009,326 EAST 4 STREET,C1,2,NaN,FAWN EAST FOURTH STREET LLC,...,0,0,0,0,0,0,0,0,0,0
2,MN,1012727501,105,11201.0,10019,33 WEST 56 STREET,RM,4,NaN,UNAVAILABLE OWNER,...,0,0,0,0,0,0,0,0,0,0
3,MN,1003730043,103,2601.0,10009,321 EAST 3 STREET,D3,3,NaN,PENTHOUSE MANAGEMENT LLC,...,0,0,0,0,0,0,0,0,0,0
4,MN,1012090028,107,181.0,10025,5 WEST 95 STREET,C5,2,NaN,"5 WEST 95TH STREET,",...,0,0,0,0,0,0,0,0,0,0


In [166]:
# left join from pluto to capture all properties
# use df for space constraints
df_pl= pluto.merge(df, how = "left", on = "bbl", indicator = True)
print(df_pl["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_pl["vacate_ind"]= np.where(
            df_pl["_merge"]=="both", 1, 0)
df_pl.loc[df_pl["_merge"]=="left_only", "num_vacate_orders"]= 0

print("\nTab of outcome var (vacate ind)")
print(df_pl[["vacate_ind", "_merge"]].value_counts())
df_pl.drop("_merge", axis = 1, inplace = True)

left_only     762424
both            2604
right_only         0
Name: _merge, dtype: int64

Tab of outcome var (vacate ind)
vacate_ind  _merge   
0           left_only    762424
1           both           2604
dtype: int64


### HPD Speculations

In [167]:
csv = drive.CreateFile({'id': '1yf86QR6yv2Z9Q6WNINW87RASUsdHPDl6'})
csv.GetContentFile('hpd_spec.csv')  

hpd_spec= pd.read_csv('hpd_spec.csv').drop("Unnamed: 0", axis = 1)
hpd_spec["bbl"]= hpd_spec["bbl"].astype(str).str.replace(".0", "", regex = False)

In [168]:
hpd_spec.head()

,bbl,speculation
0,4005150031,1
1,4155440010,1
2,4030790086,1
3,1002050023,1
4,4021390001,1


In [169]:
# left join from pluto to HPD
# use df for space constraints
df_hpd_spec= df_pl.merge(hpd_spec, how = "left", on = "bbl", indicator = True)
print(df_hpd_spec["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_hpd_spec["speculation"]= np.where(
            df_hpd_spec["_merge"]=="both", 1, 0)

df_hpd_spec.drop("_merge", axis = 1, inplace = True)

left_only     764679
both             351
right_only         0
Name: _merge, dtype: int64


In [170]:
hpd_spec.head()

,bbl,speculation
0,4005150031,1
1,4155440010,1
2,4030790086,1
3,1002050023,1
4,4021390001,1


### HPD Violations

In [171]:
csv = drive.CreateFile({'id': '1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH'})
csv.GetContentFile('hpd_viol.csv')  

hpd_viol= pd.read_csv('hpd_viol.csv').rename({"BBL":"bbl"}, axis = 1)
hpd_viol

,bbl,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio
0,3042719001,6411,5,4898
1,2024310026,4818,36,3712
2,5012720011,4170,0,2775
3,3013020001,4075,10,3257
4,2037430063,3806,55,3223
...,...,...,...,...
181588,1019430052,1,0,0
181589,1012060017,1,0,0
181590,1018850049,1,0,0
181591,1012520050,1,0,0


In [172]:
#hpd_viol= hpd_viol.drop("Unnamed: 0", axis = 1)
hpd_viol["bbl"]= hpd_viol["bbl"].astype(str)
hpd_viol= hpd_viol.drop(hpd_viol[hpd_viol["bbl"]=="0"].index, axis = 0)

hpd_viol.head()

,bbl,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio
0,3042719001,6411,5,4898
1,2024310026,4818,36,3712
2,5012720011,4170,0,2775
3,3013020001,4075,10,3257
4,2037430063,3806,55,3223


In [173]:
# left join from pluto to HPD
# use df for space constraints
df_hpd_viol= df_pl.merge(hpd_viol, how = "left", on = "bbl", indicator = True)
print(df_hpd_viol["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_hpd_viol.loc[df_hpd_viol["_merge"]=="left_only", "viol_count"]= 0

df_hpd_viol.drop("_merge", axis = 1, inplace = True)

left_only     586852
both          178176
right_only         0
Name: _merge, dtype: int64


In [174]:
df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]]= df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]].fillna(0)

In [175]:
df_hpd_viol[["viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit"]]= \
df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]].div(df_hpd_viol["unitsres"], axis = 0).fillna(0, axis = 1)
df_hpd_viol[["viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit", "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio", "unitsres"]]

,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio,unitsres
0,0.000000,0.0,0.000000,0.0,0.0,0.0,6
1,1.000000,0.0,0.600000,10.0,0.0,6.0,10
2,0.000000,0.0,0.000000,0.0,0.0,0.0,47
3,0.133333,0.0,0.000000,4.0,0.0,0.0,30
4,1.100000,0.0,0.800000,11.0,0.0,8.0,10
...,...,...,...,...,...,...,...
765023,5.500000,0.0,4.333333,33.0,0.0,26.0,6
765024,4.666667,0.0,3.333333,14.0,0.0,10.0,3
765025,1.545455,0.0,0.363636,17.0,0.0,4.0,11
765026,0.888889,0.0,0.111111,8.0,0.0,1.0,9


### DOB Building Permits

In [176]:
csv = drive.CreateFile({'id': '1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp'})
csv.GetContentFile('dev_perm.csv')  

dev= pd.read_csv('dev_perm.csv', header=[0,1], index_col=[0])['avg_permits'].reset_index()
dev["bbl"]= dev["bbl"].astype(str).str.replace(".0", "", regex = False)
dev.head()

permit_type,bbl,AL,NB
0,1000000000,0.444444,0.0
1,1000000001,1.000000,0.0
2,1000000002,0.111111,0.0
3,1000000003,0.222222,0.0
4,1000000004,0.333333,0.0


In [177]:
# left join from pluto to dev dob permits
# use df for space constraints
df_dev= df_hpd_viol.merge(dev, how = "left", on = "bbl", indicator = True)
print(df_dev["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_dev.loc[df_dev["_merge"]=="left_only", "AL"]= 0
df_dev.loc[df_dev["_merge"]=="left_only", "NB"]= 0

df_dev.drop("_merge", axis = 1, inplace = True)

left_only     742923
both           22105
right_only         0
Name: _merge, dtype: int64


### LL 84

In [178]:
csv = drive.CreateFile({'id': '1NOKtbGyL3_TBSMp_eqybVaQc67t33yuo'})
csv.GetContentFile('ll84_avg.csv')  

ll84= pd.read_csv('ll84_avg.csv').drop({"Unnamed: 0"}, axis = 1)
ll84["bbl"]= ll84["bbl"].astype(str).str.replace(".0", "", regex = False)
ll84= ll84.drop(ll84[ll84["bbl"].apply(len)<10].index, axis = 0)

In [179]:
# left join from pluto to dev dob permits
# use df for space constraints
df_ll84= df_dev.merge(ll84, how = "left", on = "bbl", indicator = True)
print(df_ll84["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_ll84.loc[df_ll84["_merge"]=="left_only", "average_energy_usage_mi"]= 1
df_ll84.loc[df_ll84["_merge"]=="both", "average_energy_usage_mi"]= 0
df_ll84.loc[df_ll84["_merge"]=="left_only", "average_energy_usage"]= 0

df_ll84.drop("_merge", axis = 1, inplace = True)

left_only     747641
both           17387
right_only         0
Name: _merge, dtype: int64


### Export analytic file (w/o census tract info)

In [180]:
df_ll84.to_csv(os.join(root, "data", "3_clean", "analysis_bbl_noacs.csv"))

In [181]:
df_ll84.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,hpd_fire_vio,hpd_b_c_vio,viol_count,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,MN,1012090027,107,181.0,10025,7 WEST 95 STREET,C2,2,NaN,ROBERT & ANNE RABBINO,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.0
1,MN,1003730016,103,2601.0,10009,326 EAST 4 STREET,C1,2,NaN,FAWN EAST FOURTH STREET LLC,...,0.0,6.0,NaN,1.000000,0.0,0.6,0.0,0.000000,0.000000,1.0
2,MN,1012727501,105,11201.0,10019,33 WEST 56 STREET,RM,4,NaN,UNAVAILABLE OWNER,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,77.366667,0.0
3,MN,1003730043,103,2601.0,10009,321 EAST 3 STREET,D3,3,NaN,PENTHOUSE MANAGEMENT LLC,...,0.0,0.0,NaN,0.133333,0.0,0.0,0.0,0.111111,0.000000,1.0
4,MN,1012090028,107,181.0,10025,5 WEST 95 STREET,C5,2,NaN,"5 WEST 95TH STREET,",...,0.0,8.0,NaN,1.100000,0.0,0.8,0.0,0.000000,0.000000,1.0


## ACS Data, for 2015-2019 5-year

In [182]:
# load acs data (to check census tract recoding)
csv = drive.CreateFile({'id': '17PvjIPesTzugausfzMi7rTiXahTiHaH1'})
csv.GetContentFile('acs_data.csv')  

acs= pd.read_csv('acs_data.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
acs= acs.loc[acs['year']==2019]
acs.head()

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhblack,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter
21721,36005000100,6864,10599.550,0.647574,6345,519,103,3322,3392,47,...,52.462121,0.262238,2.636946,0.0,0.276807,0.509907,NaN,NaN,NaN,NaN
21722,36005000200,4532,24481.190,0.185122,2193,2339,1134,862,1605,931,...,20.300088,0.000000,1.654898,0.0,1.323919,0.000000,52.459016,47.540984,11.997019,18.181818
21723,36005000400,5516,18537.140,0.297565,2810,2706,1071,1319,2362,764,...,22.915156,0.000000,2.193619,0.0,0.163162,1.341552,79.040157,20.959843,1.860921,4.906542
21724,36005001600,5825,31101.490,0.187290,2422,3403,1546,1399,1978,902,...,34.695279,0.000000,1.098712,0.0,0.858369,0.429185,12.430238,87.569762,11.415525,12.514484
21725,36005001900,3141,4877.152,0.644023,1583,1558,898,1052,1111,80,...,29.831264,0.000000,2.292264,0.0,0.700414,1.782872,10.565338,89.434662,9.638554,10.777202


In [183]:
acs['geofips_recode']= acs['geofips'].astype(str).str[2:]
acs['geofips_recode']= np.select([acs['geofips_recode'].str[0:3] == '005',
                                 acs['geofips_recode'].str[0:3] == '047',
                                 acs['geofips_recode'].str[0:3] == '061',
                                 acs['geofips_recode'].str[0:3] == '081',
                                 acs['geofips_recode'].str[0:3] == '085'],
                                ['2' + acs['geofips_recode'].str[3:],
                                '3' + acs['geofips_recode'].str[3:],
                                 '1' + acs['geofips_recode'].str[3:],
                                 '4' + acs['geofips_recode'].str[3:],
                                 '5' + acs['geofips_recode'].str[3:]])

print(acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode'].head())
acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode']= \
acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode'].str.pad(width = 7, fillchar = "0", side = "right")

acs[['geofips_recode', 'geofips']].head()

Series([], Name: geofips_recode, dtype: object)


,geofips_recode,geofips
21721,2000100,36005000100
21722,2000200,36005000200
21723,2000400,36005000400
21724,2001600,36005001600
21725,2001900,36005001900


In [192]:
acs.loc[acs["geofips_recode"]=="3114201", ]

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode
22780,36047114201,1554,13375.21,0.116185,881,673,396,282,701,175,...,0.0,2.767053,0.0,0.0,3.603604,38.271605,61.728395,8.994709,12.285714,3114201


In [184]:
df_ll84["boroct2010"]= df_ll84["boroct2010"].astype(str)

In [185]:
df_acs= pd.merge(df_ll84, acs, how = "outer", left_on =  "boroct2010", right_on = "geofips_recode", indicator = True)
df_acs.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode,_merge
0,MN,1012090027,107.0,181.0,10025.0,7 WEST 95 STREET,C2,2.0,NaN,ROBERT & ANNE RABBINO,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100,both
1,MN,1012090028,107.0,181.0,10025.0,5 WEST 95 STREET,C5,2.0,NaN,"5 WEST 95TH STREET,",...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100,both
2,MN,1012090037,107.0,181.0,10025.0,12 WEST 96 STREET,D4,3.0,NaN,12 W 96TH STREET CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100,both
3,MN,1012090049,107.0,181.0,10025.0,44 WEST 96 STREET,D4,3.0,NaN,44 OWNERS CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100,both
4,MN,1012090051,107.0,181.0,10025.0,46 WEST 96 STREET,D4,3.0,NaN,46 W 96TH ST OWNERS CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100,both


In [186]:
print(df_acs["_merge"].value_counts())
df_acs= df_acs.drop("_merge", axis = 1)

both          765023
right_only        57
left_only          5
Name: _merge, dtype: int64


## Export analytic file

In [188]:
df_acs.to_csv(os.join(root, "data", "3_clean", "analysis_bbl.csv"))

## Predictors to add?
* Number of times sold
* Sale within 2012?
* Tax Lien Sale
* Unpaid charges